# Temporal Matrix Factorization for Multivariate Time Series Forecasting

Temporal matrix factorization is an important variant in the large family of matrix factorization models. Thanks to the temporal modeling technique (for example vector autoregressive (VAR) process), temporal matrix factorization in extremely useful for multivariate time series forecasting in the presence of missing values. 
The basic idea behind the model is that matrix factorization can learn low-rank temporal patterns from partially observed time series data, while temporal modeling can capture time-evolving coefficients. 

## Matrix Factorization

Is a powerfull tool for reconstructing data matrices with missing entries. The fundamental idea is that a partially observed data matrix can be factorized into two factor matrices of relatively low-rank R. In practice, the common dynamics of large amount of time series could stem from a relativelly small number of latent temporal factors. 

Given a multivariate time series in the form of matrix with N rows and T columns:

![](https://miro.medium.com/max/700/1*UOid48ZhLXl8585baaMtow.png)

In some cases this data matrix is incomplete with some missing entries. We can define the index set of observed entries $\Omega$ and write down a matrix factorization formula with two factor matrices W and X as follows:

$$ P_{\Omega}(Y) = P_{\Omega}(W^{T}X)$$

Where W is of size R by N and X is of the size R by T. Herein, R is the low rank. We can achieve such approximation by constructing the optimization problem with a certain loss function:

$$ min_{W,X} 0.5 * \lVert P_{\Omega}(Y-W^{T}X) \rVert_{F}^2 + \rho (\lVert W \rVert_{F}^2 + \lVert X \rVert_{F}^2) $$

On time series data Y, if there exist some unobserved entries, solving such optimization problem can impute missing values. However, if we want to forecast future data points, we should build temporal correlation upon a certain time series model. There are many options for building temporal correlation on time series, including univariate autoregressive process and multivariate VAR process.

## VAR Process

The coefficient matrix in VAR allows us to capture coevolution patterns amoing multiple variables in time series data. Formally, on time series ${x_{1},x_{2},x_{3},...x_{T}}$ a dth-order VAR process can be modeled as:

$$ x_{t} \approx \sum_{k=1}^{d} A_{k}x_{t-k} $$

Until now, we have discussed the basic formulas of both matrix factorization and VAR. One question is that how to develop an elegant connection between matrix factorization and VAR?

## Temporal Matrix Factorization

### Optimization Problem

Recall that the common dynamics of large amounts of time series could stem from a relatively small number of latent temporal factors. Therefore, it is possible to model a temporal factor matrix X by a VAR process. The optimization problem of temporal matrix factorization can be formulated as follows:

$$ min_{W,X,A_{1},A_{2},...,A_{d}}  0.5 * \lVert P_{\Omega}(Y-W^{T}X) \rVert_{F}^2 \rho /2 (\lVert W \rVert_{F}^2 + \lVert X \rVert_{F}^2) + \lambda / 2 \sum_{d+1}^{T} \lVert x_{t} - \sum_{k=1}^{d} A_{k}x_{t-k} \rVert_{2}^2 $$

At first sight, this optimization problem is rather complicated and difficult to solve. To simplify the problem, we can define a sequence of temporal operators:

$$ \Psi_{k} \triangleq [O_{(T-d)*(d-k)} I_{T-d} O_{(T-d)*k}]  \in R^{T-d}*T , k = 0,1,2,...,d$$

With such notations, the optimization problem can be rewritten as follows:

$$ min_{W,X,A}  0.5 * \lVert P_{\Omega}(Y-W^{T}X) \rVert_{F}^2 \rho /2 (\lVert W \rVert_{F}^2 + \lVert X \rVert_{F}^2) + \lambda / 2 \lVert X\Psi_{0}^{T} - A(I_{d} \otimes X)\Psi^{T} \rVert_{F}^2 $$

![](https://miro.medium.com/max/700/1*CzxBxXvkWM6ES4E52VN96Q.png)

### Solution to the algorithm

We can approximate the least squares solution of both factor matrices W and X by using the conjugate [gradient method](https://medium.com/p/7f16cbae18a3). In terms of the coefficient matrix A, there is a least squares solution. Therefore we can use an **alternating minimization algorithm** to solve the optimization problem. 

In [1]:
import numpy as np

def update_cg(ver, r, q, Aq, rold):
    alpha = rold / np.inner(q,Aq)
    var = var + alpha * q
    r = r - alpha * Aq
    rnew = np.inner(r, r)
    q = r + (rnew/rold)*q
    return var, r, q, rnew

def ell_w(ind, W, X, rho):
    return X @ ((W.T @ X) * ind).T + rho * W

def conj_grad_w(sparse_mat, ind, W, X, rho, maxiter = 5):
    rank, dim1 = W.shape
    w = np.reshape(W, -1, order = 'F')
    r = np.reshape(X @ sparse_mat.T - ell_w(ind, W, X, rho), -1, order = 'F')
    q = r.copy()
    rold = np.inner(r, r)
    for it in range(maxiter):
        Q = np.reshape(q, (rank, dim1), order = 'F')
        Aq = np.reshape(ell_w(ind, Q, X, rho), -1, order = 'F')
        w, r, q, rold = update_cg(w, r, q, Aq, rold)
    return np.reshape(w, (rank, dim1), order = 'F')

def ell_x(ind, W, X, A, Psi, d, lambda0, rho):
    rank, dim2 = X.shape
    temp = np.zeros((d * rank, Psi[0].shape[0]))
    for k in range(1, d + 1):
        temp[(k - 1) * rank : k * rank, :] = X @ Psi[k].T
    temp1 = X @ Psi[0].T - A @ temp
    temp2 = np.zeros((rank, dim2))
    for k in range(d):
        temp2 += A[:, k * rank : (k + 1) * rank].T @ temp1 @ Psi[k + 1]
    return W @ ((W.T @ X) * ind) + rho * X + lambda0 * (temp1 @ Psi[0] - temp2)

def conj_grad_x(sparse_mat, ind, W, X, A, Psi, d, lambda0, rho, maxiter = 5):
    rank, dim2 = X.shape
    x = np.reshape(X, -1, order = 'F')
    r = np.reshape(W @ sparse_mat - ell_x(ind, W, X, A, Psi, d, lambda0, rho), -1, order = 'F')
    q = r.copy()
    rold = np.inner(r, r)
    for it in range(maxiter):
        Q = np.reshape(q, (rank, dim2), order = 'F')
        Aq = np.reshape(ell_x(ind, W, Q, A, Psi, d, lambda0, rho), -1, order = 'F')
        x, r, q, rold = update_cg(x, r, q, Aq, rold)
    return np.reshape(x, (rank, dim2), order = 'F')

Here, conj_grad_w is used to approximate the least squares solution to the factor matrix W. Since temporal factor matrix X is associated with both matrix factorization and VAR, approximating the least squares solution to the factor matrix X with conjugate gradient (i.e., conj_grad_x) is much more complicated than than updating W. Note that conjugate gradient is an efficient iterative algorithm, and only few iterations are required for reaching convergence.

Then, we can define the temporal operators associated with VAR as follows. The inputs are temporal dimensionality and the order of VAR, i.e., T and d. In the function, the output Psi is a Python list, and there are d + 1 arrays.

In [2]:
def generate_Psi(T, d):
    Psi = []
    for k in range(0, d + 1):
        if k == 0:
            Psi.append(np.append(np.zeros((T - d, d)), np.eye(T - d), axis = 1))
        else:
            Psi.append(np.append(np.append(np.zeros((T - d, d - k)), np.eye(T - d), axis = 1), 
                                 np.zeros((T - d, k)), axis = 1))
    return Psi

Now, we can define a function tmf for temporal matrix factorization. The inputs include sparse (or dense) time series data matrix, rank R of matrix factorization, order d of VAR, and tradeoff parameters lambda0 and rho. The outputs include an estimated time series matrix, factor matrices {W, X}, and coefficient matrix A.

In [3]:
def tmf(sparse_mat, rank, d, lambda0, rho, maxiter = 50):
    dim1, dim2 = sparse_mat.shape
    ind = sparse_mat != 0
    W = 0.01 * np.random.randn(rank, dim1)
    X = 0.01 * np.random.randn(rank, dim2)
    A = 0.01 * np.random.randn(rank, d * rank)
    Psi = generate_Psi(dim2, d)
    temp = np.zeros((d * rank, dim2 - d))
    for it in range(maxiter):
        W = conj_grad_w(sparse_mat, ind, W, X, rho)
        X = conj_grad_x(sparse_mat, ind, W, X, A, Psi, d, lambda0, rho)
        for k in range(1, d + 1):
            temp[(k - 1) * rank : k * rank, :] = X @ Psi[k].T
        A = X @ Psi[0].T @ np.linalg.pinv(temp)
        mat_hat = W.T @ X
    return mat_hat, W, X, A

In the temporal matrix factorization algorithm, we build a VAR process and perform forecasting on the temporal factor matrix. Here, we can define a function for VAR forecasting.

In [4]:
def var4cast(X, A, d, delta):
    dim1, dim2 = X.shape
    X_hat = np.append(X, np.zeros((dim1, delta)), axis = 1)
    for t in range(delta):
        X_hat[:, dim2 + t] = A @ X_hat[:, dim2 + t - np.arange(1, d + 1)].T.reshape(dim1 * d)
    return X_hat[:, - delta :]

# Forecasting Multivariate Time Series with NonStationarity Temporal Matrix Factorization

[Uber movement](https://movement.uber.com/?lang=es-PE) provides data and tools for cities to more deeply understand and address urban transportation challenges. Uber movement speed data measures hourley street speeds across a city to enable data-driven city planning and decision making. These data are indeed multivariate time series, with N road segments and T time steps (hours) and are featured as high-dimensional, sparse, and non-stationary. To overcome the challenge of predicting, we propose a nonstationary temporal matrix factorization (NoTMF) framework for multivariate time series forecasting on high dimensional and sparse Uber movement speed data. 

We consider to develop a low-rank matrix factorization framework with the following claims:

1. High-dimensional time series could stem from a relatively small number of latent temporal factors.
2. CFor the fact that modern time series usually show nonstationarity (e.g., trend, seasonality, see the following two graphics). It is necessary to consider the nonstationarity issue when constructing a low-rank matrix factorization model.

![](https://miro.medium.com/max/700/1*cpPr22FeFdyO9jflgXa5QA.png)

## Non Stationarity Temporal Matrix Factorization

To generalize the idea of temporal matrix factorization with non stationary time series, we can impose **differencing operations** (first order differencing, seasonal differencing) on temporal factors. 

$$ min_{W,X,[A_{k}]_{k=1}^{d}}  0.5 * \lVert P_{\Omega}(Y-W^{T}X) \rVert_{F}^2 \rho /2 (\lVert W \rVert_{F}^2 + \lVert X \rVert_{F}^2) + \lambda / 2 \sum_{t=d+m+1}^{T} \lVert (x_{t}-x_{t-m}) - \sum_{k=1}^{d} A_{k}(x_{t-k}-x_{t-m-k}) \rVert_{2}^2 $$


To solve the involved optimization problem, we may need to rewrite vector autoregressive process in the form of matrix. Here, we define some temporal operators associated with temporal dimensionality T, order d, and season m. These temporal operators can help reformulate the vector-form VAR as a matrix-form one.

$$ \Psi_{k} \triangleq [O_{(T-d-m)*(d-k)} I_{T-d-m} O_{(T-d-m)*(k+m)}] + [O_{(T-d-m)*(d+m-k)} I_{T-d-m} O_{(T-d-m)*k}]  \in R^{T-d-m}*T , k = 0,1,2,...,d$$

Then: 

$$ \sum_{t=d+m+1}^{T} \lVert (x_{t}-x_{t-m}) - \sum_{k=1}^{d} A_{k}(x_{t-k}-x_{t-m-k}) \rVert_{2}^2 \equiv \lVert X\Psi_{0}^{T} - \sum_{k=1}^{d} A_{k}X\Psi_{k}^{T} \rVert_{F}^2 \triangleq \lVert X\Psi_{0}^{T} - \sum_{k=1}^{d} A(I_{d} \otimes X)\Psi^{T} \rVert_{F}^2 $$

## Solution to NoTMF